In [ ]:
import sqlite3
import os

def connect():
    database_path = os.environ["MUSIC_CATALOGUE_DB"]
    connection = sqlite3.connect(database_path)
    return connection

In [7]:
from pathlib import Path
import sqlparse

def load_sql(sql_file, placeholder_values = None, show_query = False):
    # Read the query file
    query_file_path = Path("sql") / sql_file
    with open(query_file_path.absolute(), "r") as f:
        query = f.read().replace("\n", " ")

    # Replace placeholders with real values
    if placeholder_values:
        for key, value in placeholder_values.items():
            query = query.replace(f"${key}", value)

    # Show a pretty-printed form of the query
    if show_query:
        print(sqlparse.format(query, reindent=True, keyword_case="upper"))
    return query


In [ ]:
import pandas as pd

def load_genre_statistics(connection):
    query = load_sql("genre-statistics.sql")
    df = pd.read_sql_query(query, connection)
    return df